# Ensemble Model Training - Simplified Single-Branch Models

This notebook trains separate simplified CNN models for each feature type and then creates an ensemble.

In [27]:
from datetime import datetime
import os
import json
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.utils import to_categorical, Sequence
from keras.models import Model
from keras.layers import (
    Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
)
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from modules.PostgresDBHandler import PostgresDBHandler
from tqdm import tqdm
from tensorflow.keras.optimizers import Adam

In [28]:
# Configuration
dbParams = {
    "dbname": "mydatabase",
    "user": "myuser",
    "password": "mypassword",
    "host": "postgres_server",
    "port": "5432",
}

EPOCHS = 200
BATCH_SIZE = 32
KFOLD_SPLITS = 5
FIXED_LENGTH = 128

# Feature types to train models for
FEATURE_TYPES = [
    'mel_spectrogram', 'mfcc', 'chromagram', 'spectral_contrast',
    'tonnetz', 'constant_q', 'cqt', 'stft', 'harmonic_percussive', 'onset_strength'
]

# GPU configuration
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"Number of available GPUs: {len(gpus)}")
    except RuntimeError as e:
        print(e)

Number of available GPUs: 1


In [29]:
# Initialize database connection
db = PostgresDBHandler(**dbParams)
db.connect()

# Get instrument mappings
instruments_mappings = db.get_mappings_instruments()
num_classes = len(instruments_mappings)
print(f"Number of instrument classes: {num_classes}")
print("Instruments:", instruments_mappings['name'].tolist())

db.close()

Number of instrument classes: 9
Instruments: ['clarinet', 'flute', 'sax', 'cello', 'bass', 'trumpet', 'piccolo', 'oboe', 'violin']


In [30]:
dbConnect = PostgresDBHandler(**dbParams)
dbConnect.connect()
audioIDs = dbConnect.get_all_unique_audio_ids_in_processed()
processed_data = dbConnect.get_processed_fit_data(audioIDs)

all_processed_data = []
for audio_id in audioIDs:
    features = dbConnect.get_all_feature_types_for_audio(audio_id)
    feature_dict = {f['featureTypeName']: f['featurePath'] for f in features}
    instrumentID = dbConnect.get_audio_file(audio_id)['instrumentID']
    feature_dict['instrumentID'] = instrumentID
    all_processed_data.append(feature_dict)

dbConnect.close()

In [31]:
processed_df = pd.DataFrame(all_processed_data)
processed_df

,mel_spectrogram,mfcc,chromagram,spectral_contrast,tonnetz,constant_q,cqt,stft,harmonic_percussive,onset_strength,instrumentID
0,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/b5ac2081-b5...,ensemble_intermediate_results/chromagram/80e1c...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/79ce4aab...,ensemble_intermediate_results/constant_q/b566f...,ensemble_intermediate_results/cqt/1a0b026d-7ac...,ensemble_intermediate_results/stft/0f7a0f1e-cf...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/8...,9
1,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/984d1a3a-1c...,ensemble_intermediate_results/chromagram/a7b41...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/4c5fc97d...,ensemble_intermediate_results/constant_q/50290...,ensemble_intermediate_results/cqt/b7296a0c-766...,ensemble_intermediate_results/stft/7dc84afe-49...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/d...,6
2,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/8e6f7efb-36...,ensemble_intermediate_results/chromagram/59bfe...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/e3074510...,ensemble_intermediate_results/constant_q/636e6...,ensemble_intermediate_results/cqt/b0198e1f-ebb...,ensemble_intermediate_results/stft/f6ab29ba-3e...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/7...,7
3,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/cfe0a55e-ab...,ensemble_intermediate_results/chromagram/ac66a...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/d9736adf...,ensemble_intermediate_results/constant_q/a6862...,ensemble_intermediate_results/cqt/9a4c69a6-7cc...,ensemble_intermediate_results/stft/230fff4b-ef...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/7...,5
4,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/6a5c4b21-87...,ensemble_intermediate_results/chromagram/9306a...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/9f50a343...,ensemble_intermediate_results/constant_q/8cf7c...,ensemble_intermediate_results/cqt/7ebc16d1-12a...,ensemble_intermediate_results/stft/a6b747d3-34...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/1...,7
...,...,...,...,...,...,...,...,...,...,...,...
895,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/44e684c8-1c...,ensemble_intermediate_results/chromagram/27175...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/e86cca72...,ensemble_intermediate_results/constant_q/72654...,ensemble_intermediate_results/cqt/20499b08-993...,ensemble_intermediate_results/stft/1a9f47cc-1e...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/0...,3
896,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/84ae87a4-e7...,ensemble_intermediate_results/chromagram/fe1f7...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/7bc662f0...,ensemble_intermediate_results/constant_q/4f4fb...,ensemble_intermediate_results/cqt/f998362a-160...,ensemble_intermediate_results/stft/6a1a79cb-9b...,ensemble_intermediate_results/harmonic_percuss...,ensemble_intermediate_results/onset_strength/8...,2
897,ensemble_intermediate_results/mel_spectrogram/...,ensemble_intermediate_results/mfcc/5de70127-86...,ensemble_intermediate_results/chromagram/b8fd5...,ensemble_intermediate_results/spectral_contras...,ensemble_intermediate_results/tonnetz/779c6322...,ensemble_intermediate_results/constant_

In [32]:
def get_input_shape(feature_type):
    shapes = {
        'mel_spectrogram': (64, 128),
        'mfcc': (8, 128),
        'chromagram': (8, 128),
        'spectral_contrast': (3, 128),
        'tonnetz': (6, 128),
        'constant_q': (42, 128),
        'cqt': (84, 128),
        'stft': (513, 128),
        'harmonic_percussive': (1025, 128),
        'onset_strength': (1, 128)
    }
    return shapes[feature_type]

In [42]:
class SingleFeatureDataGenerator(Sequence):
    def __init__(self, df, feature_col, batch_size=32, shuffle=True, num_classes=None):
        self.df = df.reset_index(drop=True)
        self.feature_col = feature_col
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))

    def on_epoch_end(self):
        self.indices = np.arange(len(self.df))
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_df = self.df.iloc[batch_indices]

        X = []
        y = []

        for _, row in batch_df.iterrows():
            try:
                arr = np.load(row[self.feature_col])
            except Exception as e:
                print(f"Error loading {row[self.feature_col]}: {e}")
                continue
        
            if np.isnan(arr).any() or np.isinf(arr).any():
                raise ValueError(f"Feature file {row[self.feature_col]} contains NaNs or Infs.")
        
            arr = (arr - np.mean(arr)) / (np.std(arr) + 1e-8)
            if arr.ndim == 2:
                arr = np.expand_dims(arr, -1)  # shape: (H, W, 1)
        
            X.append(arr)
            y.append(row['instrumentID'])  # already label-encoded
        
        X = np.array(X)
        y = to_categorical(np.array(y), num_classes=self.num_classes)
        
        return X, y

In [51]:
def create_simple_model(input_shape, num_classes, model_name="simple_cnn"):
    input_layer = Input(shape=(*input_shape, 1), name=f"{model_name}_input")

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)

    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)

    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)

    output = Dense(num_classes, activation='softmax', name=f"{model_name}_output")(x)

    model = Model(inputs=input_layer, outputs=output, name=model_name)
    return model


In [52]:
results = {}

for feature_type in tqdm(FEATURE_TYPES, desc="Training features"):
    print(f"\n{'='*40}\nTraining model for {feature_type}\n{'='*40}")

    feature_col = feature_type
    feature_df = processed_df.dropna(subset=[feature_col])
    
    # Global label encoder
    label_encoder = LabelEncoder()
    label_encoder.fit(feature_df['instrumentID'])

    feature_df = feature_df.copy()
    feature_df['instrumentID'] = label_encoder.transform(feature_df['instrumentID'])
    num_classes = len(label_encoder.classes_)
    input_shape = get_input_shape(feature_type)

    kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
    accuracy_list, loss_list, history_list = [], [], []
    classification_reports, confusion_matrices = [], []

    for fold, (train_idx, test_idx) in enumerate(kf.split(feature_df)):
        print(f"\n--- Fold {fold+1}/{KFOLD_SPLITS} ---")
        train_df = feature_df.iloc[train_idx].reset_index(drop=True)
        test_df = feature_df.iloc[test_idx].reset_index(drop=True)

        train_df, val_df = train_test_split(
            train_df, test_size=0.2, random_state=42, stratify=train_df['instrumentID'])

        # Generators (labels are already encoded)
        train_gen = SingleFeatureDataGenerator(train_df, feature_col, BATCH_SIZE, shuffle=True, num_classes=num_classes)
        val_gen   = SingleFeatureDataGenerator(val_df,   feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)
        test_gen  = SingleFeatureDataGenerator(test_df,  feature_col, BATCH_SIZE, shuffle=False, num_classes=num_classes)

        # Model
        model = create_simple_model(input_shape, num_classes, model_name=feature_type)
        model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[early_stopping])
        history_list.append(history.history)

        # Evaluation
        loss, acc = model.evaluate(test_gen)
        loss_list.append(loss)
        accuracy_list.append(acc)
        print(f"{feature_type} - Fold {fold+1} Test accuracy: {acc:.4f}")

        # Predictions & Reports
        y_pred = model.predict(test_gen)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = []
        for _, labels in test_gen:
            y_true.extend(np.argmax(labels, axis=1))
        y_true = np.array(y_true)

        report = classification_report(y_true, y_pred_classes, output_dict=True)
        classification_reports.append(report)
        conf_matrix = confusion_matrix(y_true, y_pred_classes).tolist()
        confusion_matrices.append(conf_matrix)

        # Save model
        os.makedirs(f"models/{feature_type}", exist_ok=True)
        model.save(f"models/{feature_type}/model_fold{fold+1}.h5")

    # Save results
    results[feature_type] = {
        "accuracy_list": accuracy_list,
        "loss_list": loss_list,
        "histories": history_list,
        "classification_reports": classification_reports,
        "confusion_matrices": confusion_matrices,
    }

    with open(f"models/{feature_type}/results.json", "w") as f:
        json.dump(results[feature_type], f, indent=2)

print("\nAll training complete. Models and results saved in 'models/'")

Training features:   0%|          | 0/10 [00:00<?, ?it/s]


Training model for mel_spectrogram

--- Fold 1/5 ---
Epoch 1/200
18/18 [==============================] - 1s 14ms/step - loss: 2.1975 - accuracy: 0.1241 - val_loss: 2.1969 - val_accuracy: 0.1181
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1973 - accuracy: 0.1221 - val_loss: 2.1968 - val_accuracy: 0.1181
Epoch 3/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1972 - accuracy: 0.0926 - val_loss: 2.1966 - val_accuracy: 0.1181
Epoch 4/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1973 - accuracy: 0.1170 - val_loss: 2.1965 - val_accuracy: 0.1181
Epoch 5/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1961 - accuracy: 0.1533 - val_loss: 2.1964 - val_accuracy: 0.1181
Epoch 6/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1980 - accuracy: 0.0984 - val_loss: 2.1965 - val_accuracy: 0.1181
Epoch 7/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1973 - accuracy: 0

/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/200
18/18 [==============================] - 1s 15ms/step - loss: 2.1972 - accuracy: 0.1026 - val_loss: 2.1969 - val_accuracy: 0.1181
Epoch 2/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1976 - accuracy: 0.1121 - val_loss: 2.1968 - val_accuracy: 0.1181
Epoch 3/200
18/18 [==============================] - 0s 11ms/step - loss: 2.1969 - accuracy: 0.1186 - val_loss: 2.1967 - val_accuracy: 0.1181
Epoch 4/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1988 - accuracy: 0.0872 - val_loss: 2.1967 - val_accuracy: 0.1181
Epoch 5/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1970 - accuracy: 0.1169 - val_loss: 2.1966 - val_accuracy: 0.1181
Epoch 6/200
18/18 [==============================] - 0s 10ms/step - loss: 2.1964 - accuracy: 0.1144 - val_loss: 2.1966 - val_accuracy: 0.1181
Epoch 7/200
 8/18 [============>.................] - ETA: 0s - loss: 2.1997 - accuracy: 0.0896

Training features:   0%|          | 0/10 [00:08<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# Create ensemble predictions
print("\n" + "="*50)
print("Creating Ensemble Predictions")
print("="*50)

# Use the last fold of each feature type for ensemble evaluation
ensemble_results = {
    'accuracy_list': [],
    'loss_list': [],
    'classification_reports': [],
    'confusion_matrices': []
}

# For simplicity, we'll use the last fold of each feature type
for fold in tqdm(range(KFOLD_SPLITS), desc = "Ensemble Folds", Leave = True):
    print(f"\n--- Ensemble Fold {fold + 1}/{KFOLD_SPLITS} ---")
    
    # Get predictions from all models for this fold
    all_predictions = {}
    
    for feature_type in available_feature_types:
        if feature_type in all_results:
            # Get the model from this fold
            model = all_results[feature_type]['models'][fold]
            
            # Get test data for this fold (we need to recreate it)
            df = df_dict[feature_type]
            kf = KFold(n_splits=KFOLD_SPLITS, shuffle=True, random_state=42)
            train_idx, test_idx = list(kf.split(df))[fold]
            test_df = df.iloc[test_idx].reset_index(drop=True)
            
            test_generator = SingleFeatureDataGenerator(test_df, feature_type, batch_size=BATCH_SIZE, shuffle=False)
            
            # Get predictions
            pred = model.predict(test_generator, verbose=0)
            all_predictions[feature_type] = pred
            
            # Store true labels (should be the same for all feature types)
            if 'y_true' not in locals():
                y_true = test_generator.get_labels()
    
    # Simple averaging ensemble
    if all_predictions:
        ensemble_pred = np.mean(list(all_predictions.values()), axis=0)
        ensemble_pred_classes = np.argmax(ensemble_pred, axis=1)
        
        # Calculate ensemble accuracy
        ensemble_accuracy = accuracy_score(y_true, ensemble_pred_classes)
        ensemble_results['accuracy_list'].append(ensemble_accuracy)
        
        print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")
        
        # Classification report
        report = classification_report(y_true, ensemble_pred_classes, output_dict=True)
        ensemble_results['classification_reports'].append(report)
        
        # Confusion matrix
        conf_matrix = confusion_matrix(y_true, ensemble_pred_classes).tolist()
        ensemble_results['confusion_matrices'].append(conf_matrix)

# Store ensemble results
all_results['ensemble'] = ensemble_results

In [ ]:
# Save results and models
try:
    os.mkdir("ensemble_models")
except FileExistsError:
    print("Folder already exists")
except Exception:
    print("Unknown error")

# Create version folder
date_part = datetime.now().date().__str__().replace('-', '_')
last_version = os.listdir(path="ensemble_models") if os.path.exists("ensemble_models") else []
last_version = [name.rpartition("_v")[-1] for name in last_version if date_part in name]
if len(last_version):
    last_version = int(sorted(last_version)[-1])
else:
    last_version = 0
folder_name = f"{date_part}_v{last_version+1}"

os.makedirs(os.path.join("ensemble_models", folder_name), exist_ok=True)

# Save individual models
for feature_type, model in all_models.items():
    model_path = os.path.join("ensemble_models", folder_name, f"{feature_type}_model.h5")
    model.save(model_path)
    print(f"Saved {feature_type} model to {model_path}")

# Save results
results_data = {
    'individual_results': {ft: {k: v for k, v in res.items() if k != 'models'} 
                          for ft, res in all_results.items() if ft != 'ensemble'},
    'ensemble_results': all_results['ensemble'],
    'feature_types': available_feature_types,
    'num_classes': num_classes,
    'feature_shapes': FEATURE_SHAPES,
    'training_config': {
        'epochs': EPOCHS,
        'batch_size': BATCH_SIZE,
        'kfold_splits': KFOLD_SPLITS,
        'fixed_length': FIXED_LENGTH
    },
    'instrument_mappings': instruments_mappings.to_dict()
}

results_path = os.path.join("ensemble_models", folder_name, "results.json")
with open(results_path, 'w') as f:
    json.dump(results_data, f, indent=2, default=str)

print(f"\nResults saved to: {results_path}")
print(f"Models saved to: ensemble_models/{folder_name}/")

In [ ]:
# Print summary of results
print("\n" + "="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\nIndividual Model Performance:")
for feature_type in available_feature_types:
    if feature_type in all_results:
        accuracies = all_results[feature_type]['accuracy_list']
        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)
        print(f"  {feature_type}: {mean_acc:.4f} ± {std_acc:.4f}")

print("\nEnsemble Performance:")
if 'ensemble' in all_results:
    ensemble_accuracies = all_results['ensemble']['accuracy_list']
    ensemble_mean = np.mean(ensemble_accuracies)
    ensemble_std = np.std(ensemble_accuracies)
    print(f"  Ensemble: {ensemble_mean:.4f} ± {ensemble_std:.4f}")

# Find best individual model
best_individual = max(
    [(ft, np.mean(all_results[ft]['accuracy_list'])) 
     for ft in available_feature_types if ft in all_results],
    key=lambda x: x[1]
)

improvement = ensemble_mean - best_individual[1]
print(f"\nBest Individual Model: {best_individual[0]} ({best_individual[1]:.4f})")
print(f"Ensemble Improvement: {improvement:.4f} ({improvement*100:.2f}%)")